# PreProcessing: Car sales dataframe
_________
**Introduction:**<br>
The purpose of this exercise is to analyze used car sales data from the United States and produce a web app to share findings. First, the data will be loaded, then opened and inspected in order to understand the structure and potential issues. Next is processing the data. By making necessary or useful updates, the data will be prepared for further data exploration.

Preprocessing the data will consist of two parts. The first part is cleaning the data. This will ensure the data is accurate and consistent. Techniques involved are:

Filling Missing Values: Data points that are missing will be filled in to avoid errors. For quantitative data, an average from similiar data will be utilized; for qualitative data points, the most common entry among similiar data.

Standardizing Values: Data entries may have different spelling or formats from eachother while meaning to express the same value. An example that comes up is (f-150, f150, f_150 superduty)

Correcting Data Types: Ensure each column of data is in the correct format, such that numbers are numbers, and dates are dates.

Feature Engineering: Creation of new data columns that might be useful, like extracting the manufacturer from a model column.

Most of the preprocessing will be done in the preprocessing notebook ('PreProcessing.ipynb'). Converting dates to datetime format as well as the creation of a second dataframe storing sales data, will be conducted in the EDA notebook ('EDA.ipynb')

At the end of preprocessing, the state of the data has been saved as: processed_vehicles.csv
After the creation of the sales dataframe, it was stored in the root directory as: vehicle_sales.csv
The original data file is located in the root directory and is titled 'vehicles_us.csv'


Tools being used during the analysis are:

Pandas: <br>
Library with tools used for data manipulation and analysis. All tables being displayed are in the pandas dataframe format. This is the primary tool used for cleaning, transforming, and aggregating the dataset.

NumPy: <br>
A library designed to handles numerical support across larger datasets.

Streamlit:<br>
A library used to build web applications with an emphasis on data science. Streamlit allows for creation of interactive applications to share data insights

Plotly Express:<br>
A data visualization library used to create interactive plots and visualizations.

Datetime:<br>
Datetime is a module used to interupt, handle, and change date or time data.

Render:<br>
A web application hosting site. Render will be linked to a gitHub account hosting this project/ repository.

GitHub:<br>
A web-based platform that allows users to store, share, and manage code. Github is also a service that facilitates version control, allowing multiple contributers to make changes, while offering the means to correct potential mistakes.

VScode:<br>
IDE or coding platform used to in creation of this project.

I have separated the pre processing section, to standardize string values there is a function that requires user imports to create mapping. Re-running all cells will leave you stuck in the middle.
 **Pre-Processing**
___
cleaning done: 
standardize string columns
missing values: 
model_year dropped
odometer filled with averages from similiar car, year, and condition
paint filled with unknown
cylinder is actually a qualitative, we will fill with an unrealistic value -999. 
is_4wd is filled with 0's,
'make' column creation

In [34]:
# import pandas
import pandas as pd

In [35]:
# load the data
ve = pd.read_csv('../vehicles_us.csv')

In [37]:
# take a look at the data
print(ve.info())
ve.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB
None


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


#### Price column
___
No missing values, numeric data


In [38]:
# view prices
ve['price'].describe()

count     51525.000000
mean      12132.464920
std       10040.803015
min           1.000000
25%        5000.000000
50%        9000.000000
75%       16839.000000
max      375000.000000
Name: price, dtype: float64

In [39]:

ve['price'].value_counts()

price
1        798
6995     719
5995     655
4995     624
3500     620
        ... 
58500      1
3993       1
32987      1
3744       1
7455       1
Name: count, Length: 3443, dtype: int64

Price column looks fine. $1 cars seems weird, especially how many there are, however I am unable to say they aren't correct. Maybe this is from an auction house.
____

#### Model_year column
___
Contains missing values, numeric qualitative value

In [40]:
# model_year column
ve['model_year'].describe()

count    47906.000000
mean      2009.750470
std          6.282065
min       1908.000000
25%       2006.000000
50%       2011.000000
75%       2014.000000
max       2019.000000
Name: model_year, dtype: float64

Our latest data is 2019, most (25%-100%) percentiles are the last 20 years, standard deviation is 6 years, mean is less than the median so there are old vehicle outliers or the distribution skews left (older). 

In [41]:
# what is going on with 1908 cars?
ve[ve['model_year'] == 1908]

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
33906,12995,1908.0,gmc yukon,good,8.0,gas,169328.0,automatic,SUV,black,NaN,2018-07-06,34
33907,12995,1908.0,cadillac escalade,excellent,8.0,gas,NaN,automatic,SUV,white,NaN,2018-06-24,25


These entries are definitely wrong, as those vehicles didn't exist in 1908. The Indices are actually right next to eachother they could have been entered by the same person.

In [42]:
# view of the vehicle models
# qualitative filling is done by most common result, or mode
print(f"Fill incorrect escalade values with: {
      ve[ve['model'] == 'cadillac escalade']['model_year'].mode()}")

ve[ve['model'] == 'cadillac escalade']

Fill incorrect escalade values with: 0    2007.0
Name: model_year, dtype: float64


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
215,12500,2009.0,cadillac escalade,excellent,8.0,gas,159000.0,automatic,SUV,NaN,1.0,2018-10-01,26
610,45500,2016.0,cadillac escalade,excellent,8.0,gas,37874.0,automatic,SUV,NaN,1.0,2018-05-14,20
679,40995,2015.0,cadillac escalade,like new,8.0,gas,55218.0,automatic,pickup,NaN,1.0,2018-10-17,16
1285,7000,2005.0,cadillac escalade,excellent,8.0,gas,194300.0,automatic,SUV,black,NaN,2019-01-31,13
2020,30000,2016.0,cadillac escalade,excellent,8.0,gas,NaN,automatic,SUV,black,NaN,2018-10-10,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50501,10799,2007.0,cadillac escalade,excellent,8.0,gas,132900.0,automatic,truck,white,1.0,2018-11-10,26
50664,76995,2018.0,cadillac escalade,excellent,8.0,gas,4500.0,other,SUV,white,1.0,2018-10-25,21
50791,14995,NaN,cadillac escalade,good,8.0,gas,161162.0,automatic,SUV,NaN,NaN,2018-05-06,33
51361,8500,2008.0,cadillac escalade,excellent,8.0,gas,200538.0,automatic,SUV,NaN,1.0,2019-04-09,43


In [43]:
# develop a plan to fill missing model_year's

# group vehicles by both model and year, showing how many are available for each combination, their average price and thow many are missing
grouped = ve.groupby(['model', 'model_year'], dropna=False).agg(mean=('price', 'mean'),
                                                                count=(
                                                                    'price', 'count'),
                                                                missing=('model_year', lambda x: x.isnull().sum())).reset_index()

# sort the group by missing values so I know where to start
grouped = grouped.sort_values('missing', ascending=False)
print(grouped)

                         model  model_year          mean  count  missing
751                 ford f-150         NaN  13619.738220    191      191
350   chevrolet silverado 1500         NaN  14992.625806    155      155
1911                  ram 1500         NaN  13865.521368    117      117
315        chevrolet silverado         NaN  13816.051020     98       98
1939                  ram 2500         NaN  20586.200000     80       80
...                        ...         ...           ...    ...      ...
785                 ford f-250      2008.0  12759.107143     28        0
784                 ford f-250      2007.0  11794.900000     10        0
783                 ford f-250      2006.0  10179.421053     38        0
782                 ford f-250      2005.0   9168.650000     20        0
1163               ford ranger      1992.0    800.000000      1        0

[2326 rows x 5 columns]


We will come back to model_year, let's clean up model first
___
#### Model column
___
model has both make and model information in it. There are also non-standard naming conventions

In [44]:
# Create a new column 'make' by extracting the first word from 'model'
ve['make'] = ve['model'].str.split(' ').str[0]

# View the new column
ve['make'].unique()

array(['bmw', 'ford', 'hyundai', 'chrysler', 'toyota', 'honda', 'kia',
       'chevrolet', 'ram', 'gmc', 'jeep', 'nissan', 'subaru', 'dodge',
       'mercedes-benz', 'acura', 'cadillac', 'volkswagen', 'buick'],
      dtype=object)

In [45]:
# Return the model column back to strings

# We split on spaces, and want to drop the make information that is now in another column
ve['model'] = ve['model'].str.split(' ').str[1:]

# Join remaining model information back together on spaces
ve['model'] = ve['model'].apply(' '.join)

In [46]:
# How dirty is the data
ve[ve['make'] == 'ford']['model'].unique()

array(['f-150', 'fusion se', 'focus', 'f150 supercrew cab xlt', 'mustang',
       'f-250 sd', 'f250 super duty', 'mustang gt coupe 2d', 'explorer',
       'f-350 sd', 'edge', 'f-250', 'f150', 'expedition', 'taurus',
       'f350 super duty', 'ranger', 'escape', 'fusion',
       'f-250 super duty', 'focus se', 'f250', 'f350', 'econoline'],
      dtype=object)

In [47]:
# standardize names for a column function
def standardize_model_names(dataframe):
    """
    Standardizes data entries based on user input for mapping. There are user guided prompts to assist.

    :param dataframe: pd.DataFrame - The DataFrame you are cleaning.
    :return: pd.DataFrame - The same dataframe with updated values.
    """
    # Print available columns to the user
    columns = dataframe.columns
    print("Available columns:", columns)

    try:
        # Ask user to input the column they want to standardize
        input_column = input(
            "Enter the column you wish to standardize: ").strip()
        if input_column not in columns:
            raise ValueError(
                "Invalid column selected. Please enter a column from the list.")

        # Extract input column
        unique_options = dataframe[input_column].unique()
        print(f"{input_column}'s unique options:", unique_options)

        # Ask user to input the input_column they want to standardize
        choice_made = input(
            f"Enter the {input_column} you wish to standardize (as seen in list below): ").strip().lower()
        if choice_made not in unique_options:
            raise ValueError(
                "Invalid selection. Please enter an option from the list.")

         # Print available columns
        print("Available features:", columns)

        # Ask user to declare what feature they want to standardize
        feature = input(
            f"Enter the feature you wish to standardize (as seen in list below): ").strip().lower()
        if feature not in columns:
            raise ValueError(
                "Invalid feature. Please enter an option from the list.")

        # Filter models for the chosen make
        filtered_models = dataframe[dataframe[input_column]
                                    == choice_made][feature].unique()
        print(f"Unique models for {feature}:", filtered_models)

        # Store original models for comparison
        original_models = dataframe[dataframe[input_column]
                                    == choice_made][feature].copy()

        # Construct the mapping based on user input
        model_mapping = {}
        for model in filtered_models:
            standard_name = input(f"Enter the standardized name for '{
                                  model}' (or press Enter to keep as is): ").strip().lower()
            if standard_name:
                model_mapping[model] = standard_name

        # Replace non-standard names with standardized ones
        dataframe[feature] = dataframe[feature].replace(
            model_mapping, regex=True)

        # Show changes
        new_models = dataframe[dataframe[input_column]
                               == choice_made][feature]
        changes = original_models != new_models
        print("Changes made:")
        for original, new in zip(original_models[changes], new_models[changes]):
            print(f"'{original}' -> '{new}'")

    except ValueError as e:
        print(e)

    return dataframe

In [49]:
# run the function on each column that needs attention
# bmw
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'ram' 'gmc' 'jeep' 'nissan' 'subaru' 'dodge' 'mercedes-benz' 'acura'
 'cadillac' 'volkswagen' 'buick']
Invalid selection. Please enter an option from the list.


In [50]:
# run the function on each column that needs attention
# ford
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'ram' 'gmc' 'jeep' 'nissan' 'subaru' 'dodge' 'mercedes-benz' 'acura'
 'cadillac' 'volkswagen' 'buick']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for model: ['f-150' 'fusion se' 'focus' 'f150 supercrew cab xlt' 'mustang' 'f-250 sd'
 'f250 super duty' 'mustang gt coupe 2d' 'explorer' 'f-350 sd' 'edge'
 'f-250' 'f150' 'expedition' 'taurus' 'f350 super duty' 'ranger' 'escape'
 'fusion' 'f-250 super duty' 'focus se' 'f250' 'f350' 'econoline']
Changes made:
'fusion se' -> 'fusio

In [16]:
# run the function on each column that needs attention
# hyundai
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'ram' 'gmc' 'jeep' 'nissan' 'subaru' 'dodge' 'mercedes-benz' 'acura'
 'cadillac' 'volkswagen' 'buick']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for model: ['sonata' 'elantra' 'santa fe']
Changes made:


In [17]:
# run the function on each column that needs attention
# chrysler
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'ram' 'gmc' 'jeep' 'nissan' 'subaru' 'dodge' 'mercedes-benz' 'acura'
 'cadillac' 'volkswagen' 'buick']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for model: ['200' '300' 'town & country']
Changes made:


In [51]:
# run the function on each column that needs attention
# toyota
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'ram' 'gmc' 'jeep' 'nissan' 'subaru' 'dodge' 'mercedes-benz' 'acura'
 'cadillac' 'volkswagen' 'buick']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for model: ['camry' 'rav4' 'camry le' 'highlander' 'tacoma' '4runner' 'corolla'
 'prius' 'tundra' 'sienna']
Changes made:
'camry le' -> 'camry'
'camry le' -> 'camry'
'camry le' -> 'camry'
'camry le' -> 'camry'
'camry le' -> 'camry'
'camry le' -> 'camry'
'camry le' -> 'camry'
'camry le' -> 'camry'
'camry le' -> 'camry'
'camry le' -

In [52]:
# run the function on each column that needs attention
# honda
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'ram' 'gmc' 'jeep' 'nissan' 'subaru' 'dodge' 'mercedes-benz' 'acura'
 'cadillac' 'volkswagen' 'buick']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for model: ['pilot' 'accord' 'cr-v' 'civic' 'civic lx' 'odyssey']
Changes made:
'civic lx' -> 'civic'
'civic lx' -> 'civic'
'civic lx' -> 'civic'
'civic lx' -> 'civic'
'civic lx' -> 'civic'
'civic lx' -> 'civic'
'civic lx' -> 'civic'
'civic lx' -> 'civic'
'civic lx' -> 'civic'
'civic lx' -> 'civic'
'civic lx' -> 'civic'
'civic lx'

In [20]:
# run the function on each column that needs attention
# kia
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'ram' 'gmc' 'jeep' 'nissan' 'subaru' 'dodge' 'mercedes-benz' 'acura'
 'cadillac' 'volkswagen' 'buick']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for model: ['sorento' 'soul']
Changes made:


In [54]:
# run the function on each column that needs attention
# chevrolet
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'ram' 'gmc' 'jeep' 'nissan' 'subaru' 'dodge' 'mercedes-benz' 'acura'
 'cadillac' 'volkswagen' 'buick']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for model: ['silverado_1500' 'traverse' 'tahoe' 'malibu' 'impala' 'corvette'
 'equinox' 'colorado' 'camaro' 'cruze' 'silverado_1500 3500hd'
 'silverado_1500 crew' 'suburban' 'silverado_1500 2500hd' 'trailblazer']
Changes made:
'silverado_1500 3500hd' -> 'silverado_3500'
'silverado_1500 crew' -> 'silverado_1500'
'silverado_1500 cre

In [55]:
# run the function on each column that needs attention
# ram, ram is a sub group of dodge
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
transmission's unique options: ['automatic' 'manual' 'other']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for make: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'ram' 'gmc' 'jeep' 'nissan' 'subaru' 'dodge' 'mercedes-benz' 'acura'
 'cadillac' 'volkswagen' 'buick']
Changes made:
'ram' -> 'dodge'
'ram' -> 'dodge'
'ram' -> 'dodge'
'ram' -> 'dodge'
'ram' -> 'dodge'
'ram' -> 'dodge'
'ram' -> 'dodge'
'ram' -> 'dodge'
'ram' -> 'dodge'
'ram' -> 'dodge'
'ram' -> 'dodge'
'ram' -> 'dodge'
'ram' -> 'dodge'
'ram' -> 'dodge'
'ram' -> 'dodge'
'ram' -> 'dodg

In [56]:
# run ram again but this time correcting the 'make' column
# ram, ram is a sub group of dodge
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'dodge' 'gmc' 'jeep' 'nissan' 'subaru' 'mercedes-benz' 'acura' 'cadillac'
 'volkswagen' 'buick']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for model: ['1500' 'charger' 'grand caravan' '3500' '2500' 'dakota']
Changes made:
'1500' -> 'ram_1500'
'1500' -> 'ram_1500'
'1500' -> 'ram_1500'
'1500' -> 'ram_1500'
'1500' -> 'ram_1500'
'1500' -> 'ram_1500'
'1500' -> 'ram_1500'
'1500' -> 'ram_1500'
'1500' -> 'ram_1500'
'3500' -> 'ram_3500'
'2500' -> 'ram_2500'
'3500' -> 'ram_3500'
'25

In [57]:
# run the function on each column that needs attention
# gmc
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'dodge' 'gmc' 'jeep' 'nissan' 'subaru' 'mercedes-benz' 'acura' 'cadillac'
 'volkswagen' 'buick']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for model: ['yukon' 'sierra' 'sierra ram_2500hd' 'sierra ram_1500' 'acadia']
Changes made:
'sierra ram_2500hd' -> 'sierra_2500'
'sierra ram_1500' -> 'sierra_1500'
'sierra ram_1500' -> 'sierra_1500'
'sierra ram_2500hd' -> 'sierra_2500'
'sierra ram_2500hd' -> 'sierra_2500'
'sierra ram_1500' -> 'sierra_1500'
'sierra ram_1500' -> 'sierra_15

In [58]:
# run the function on each column that needs attention
# jeep
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'dodge' 'gmc' 'jeep' 'nissan' 'subaru' 'mercedes-benz' 'acura' 'cadillac'
 'volkswagen' 'buick']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for model: ['cherokee' 'wrangler' 'liberty' 'grand cherokee' 'grand cherokee laredo'
 'wrangler unlimited']
Changes made:
'grand cherokee laredo' -> 'grand cherokee'
'grand cherokee laredo' -> 'grand cherokee'
'wrangler unlimited' -> 'wrangler'
'wrangler unlimited' -> 'wrangler'
'wrangler unlimited' -> 'wrangler'
'grand cherokee laredo'

In [59]:
# run the function on each column that needs attention
# nissan
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'dodge' 'gmc' 'jeep' 'nissan' 'subaru' 'mercedes-benz' 'acura' 'cadillac'
 'volkswagen' 'buick']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for model: ['altima' 'rogue' 'frontier crew cab sv' 'versa' 'maxima' 'sentra'
 'frontier' 'murano']
Changes made:
'frontier crew cab sv' -> 'frontier'
'frontier crew cab sv' -> 'frontier'
'frontier crew cab sv' -> 'frontier'
'frontier crew cab sv' -> 'frontier'
'frontier crew cab sv' -> 'frontier'
'frontier crew cab sv' -> 'frontier'
'f

In [61]:
# run the function on each column that needs attention
# subaru
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'dodge' 'gmc' 'jeep' 'nissan' 'subaru' 'mercedes-benz' 'acura' 'cadillac'
 'volkswagen' 'buick']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for model: ['outback' 'impreza' 'forester']
Changes made:


In [62]:
# run the function on each column that needs attention
# mercedes-benz
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'dodge' 'gmc' 'jeep' 'nissan' 'subaru' 'mercedes-benz' 'acura' 'cadillac'
 'volkswagen' 'buick']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for model: ['benze sprinter ram_2500']
Changes made:
'benze sprinter ram_2500' -> 'sprinter'
'benze sprinter ram_2500' -> 'sprinter'
'benze sprinter ram_2500' -> 'sprinter'
'benze sprinter ram_2500' -> 'sprinter'
'benze sprinter ram_2500' -> 'sprinter'
'benze sprinter ram_2500' -> 'sprinter'
'benze sprinter ram_2500' -> 'sprinter'
'benz

In [63]:
# run the function on each column that needs attention
# acura
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'dodge' 'gmc' 'jeep' 'nissan' 'subaru' 'mercedes-benz' 'acura' 'cadillac'
 'volkswagen' 'buick']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for model: ['tl']
Changes made:


In [64]:
# run the function on each column that needs attention
# cadillac
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'dodge' 'gmc' 'jeep' 'nissan' 'subaru' 'mercedes-benz' 'acura' 'cadillac'
 'volkswagen' 'buick']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for model: ['escalade']
Changes made:


In [65]:
# run the function on each column that needs attention
# volkswagen
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'dodge' 'gmc' 'jeep' 'nissan' 'subaru' 'mercedes-benz' 'acura' 'cadillac'
 'volkswagen' 'buick']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for model: ['jetta' 'passat']
Changes made:


In [66]:
# run the function on each column that needs attention
# buick
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
make's unique options: ['bmw' 'ford' 'hyundai' 'chrysler' 'toyota' 'honda' 'kia' 'chevrolet'
 'dodge' 'gmc' 'jeep' 'nissan' 'subaru' 'mercedes-benz' 'acura' 'cadillac'
 'volkswagen' 'buick']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for model: ['enclave']
Changes made:


Now all the vehicle models are standardized
___
#### Model_year part 2
___
we need to fill missing model_year
model_year is a qualitative variable so we will use mode

In [ ]:
# filling qualitative variables with the most common entry

# Group by the specified columns and calculate the mode for the missing column
# loop through dataframe
for index in range(len(ve)):

    # for each index(row), if model_year is missing
    if pd.isna(ve.iloc[index, 1]):

        # Calculate the most common model_year for the car model at the current index
        most_common_year = ve[ve['model'] ==
                              ve.iloc[index, 2]]['model_year'].mode()

        # If a mode exists, fill the missing value
        if not most_common_year.empty:
            ve.iloc[index, 1] = most_common_year.iloc[0]

In [68]:
# are they filled?
print(ve['model_year'].isna().sum())
ve.head()

0


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
0,9400,2011.0,x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19,bmw
1,25500,2014.0,f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50,ford
2,5500,2013.0,sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79,hyundai
3,1500,2003.0,f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9,ford
4,14900,2017.0,200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28,chrysler


In [69]:
# all the Nan's are filled now are there outliers?
ve[ve['model_year'] < 1960]

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
10018,23900,1955.0,f-250,excellent,6.0,gas,47180.0,manual,truck,blue,NaN,2018-12-22,61,ford
14752,15000,1954.0,f-150,excellent,NaN,gas,3565.0,manual,pickup,black,NaN,2019-02-16,13,ford
22595,21000,1948.0,impala,like new,8.0,gas,4000.0,automatic,sedan,red,NaN,2019-01-18,24,chevrolet
33906,12995,1908.0,yukon,good,8.0,gas,169328.0,automatic,SUV,black,NaN,2018-07-06,34,gmc
33907,12995,1908.0,escalade,excellent,8.0,gas,NaN,automatic,SUV,white,NaN,2018-06-24,25,cadillac
34713,5000,1936.0,f-150,excellent,6.0,gas,30000.0,manual,pickup,purple,NaN,2018-11-22,10,ford
36582,44900,1949.0,suburban,good,NaN,gas,1800.0,automatic,wagon,orange,NaN,2018-08-19,10,chevrolet
39580,35000,1958.0,impala,excellent,8.0,gas,3184.0,automatic,coupe,black,NaN,2018-05-19,33,chevrolet
45694,18000,1929.0,f-150,good,8.0,gas,NaN,manual,other,silver,NaN,2018-11-18,59,ford
48414,37900,1958.0,impala,good,8.0,gas,62799.0,automatic,coupe,NaN,NaN,2018-08-11,10,chevrolet


From the internet: <br>
ford's f series didnt exist until 1948<br>
impala started in 1958 <br>
yukon was 1992 <br>
escalade was 1998<br>
suburban was 1935<br>


In [70]:
# impala update
ve.iloc[22595, 1] = 1958

# yukon
ve.iloc[33906, 1] = 1992

# escalade
ve.iloc[33907, 1] = 1998

# ford f series
ve.iloc[34713, 1] = 1948

#### Condition column
____
no missing values, string date type

In [71]:
ve['condition'].value_counts(dropna=False)

condition
excellent    24773
good         20145
like new      4742
fair          1607
new            143
salvage        115
Name: count, dtype: int64

condition column looks great
___
#### Cylinders column
____
has missing values, they are again qualitative

In [72]:
ve['cylinders'].value_counts(dropna=False)

cylinders
8.0     15844
6.0     15700
4.0     13864
NaN      5260
10.0      549
5.0       272
3.0        34
12.0        2
Name: count, dtype: int64

About 10% of data is missing. We might not even use this column but we will fill the same method as model_year

In [73]:
# filling qualitative variables with the most common entry

# Group by the specified columns and calculate the mean for the missing column
# loop through dataframe
for index in range(len(ve)):

    # for each index(row), if cylinders is missing
    if pd.isna(ve.iloc[index, 4]):

        # Calculate the most common cylinder for the car model and year at the current index
        most_common_cyl = ve[(ve['model'] == ve.iloc[index, 2]) &
                             (ve['model_year'] == ve.iloc[index, 1])]['cylinders'].mode()

        # If a mode exists, fill the missing value
        if not most_common_cyl.empty:
            ve.iloc[index, 4] = most_common_cyl.iloc[0]

In [74]:
print(ve['cylinders'].isna().sum())
ve['cylinders'].value_counts(dropna=False)

20


cylinders
8.0     17606
6.0     17534
4.0     15472
10.0      563
5.0       294
3.0        34
NaN        20
12.0        2
Name: count, dtype: int64

In [75]:
ve[ve['cylinders'].isna()]

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
1101,9200,1975.0,f-150,excellent,NaN,gas,NaN,automatic,truck,green,NaN,2018-08-28,40,ford
3844,1900,1986.0,sentra,excellent,NaN,gas,NaN,manual,coupe,blue,NaN,2018-10-29,63,nissan
6982,3900,1977.0,suburban,fair,NaN,gas,NaN,automatic,SUV,custom,NaN,2019-02-02,71,chevrolet
7113,84900,2019.0,f-350,new,NaN,diesel,9000.0,automatic,pickup,white,1.0,2019-03-26,7,ford
7121,6900,2009.0,cherokee,excellent,NaN,gas,130023.0,automatic,SUV,black,1.0,2018-09-03,19,jeep
11087,16000,1971.0,camaro,excellent,NaN,gas,NaN,manual,coupe,brown,NaN,2018-12-28,125,chevrolet
12760,1050,1995.0,corolla,fair,NaN,gas,428000.0,manual,sedan,black,NaN,2018-06-13,12,toyota
14752,15000,1954.0,f-150,excellent,NaN,gas,3565.0,manual,pickup,black,NaN,2019-02-16,13,ford
15623,5300,2005.0,charger,excellent,NaN,gas,125000.0,automatic,sedan,red,NaN,2018-10-30,46,dodge
27075,3800,1988.0,ram_2500,good,NaN,gas,64000.0,automatic,pickup,NaN,1.0,2018-08-26,34,dodge


In [76]:
# let's run cylinders again this time simply with model

# Group by the specified columns and calculate the mean for the missing column
# loop through dataframe
for index in range(len(ve)):

    # for each index(row), if cylinders is missing
    if pd.isna(ve.iloc[index, 4]):

        # Calculate the most common cylinder for the car model and year at the current index
        most_common_cyl = ve[(
            ve['model'] == ve.iloc[index, 2])]['cylinders'].mode()

        # If a mode exists, fill the missing value
        if not most_common_cyl.empty:
            ve.iloc[index, 4] = most_common_cyl.iloc[0]

In [77]:
# any Nan's left?
ve[ve['cylinders'].isna()]

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make


Great! slightly more 8/6/4/ and a few more 10 and 5 cylinder 
___
#### Fuel column
___
No missing values

In [78]:
ve['fuel'].value_counts()

fuel
gas         47288
diesel       3714
hybrid        409
other         108
electric        6
Name: count, dtype: int64

Looks great
___
#### Odometer column
___
Missing values, numerical and quantitative

In [79]:
# function to fill quantitative variables using the mean of filtered down subset

def fill_missing(col_1, col_2, col_3, col_4, df=ve):
    """ 
    This function is designed to fill missing quantitative variables.

    We will be filtering for multiple columns based on index

    And using the average odometer for vehicles matching these 3 columns

    call .info() on your df to gain the index for which columns you would like to filter for

    """
    # Group by the specified columns and calculate the mean for the missing column
    average_odometer = df.groupby([df.columns[col_1], df.columns[col_2], df.columns[col_3]])[
        df.columns[col_4]].mean().reset_index()

    # Merge the averages back onto the original DataFrame
    merged_df = pd.merge(df, average_odometer, on=[df.columns[col_1], df.columns[col_2], df.columns[col_3]],
                         suffixes=('', '_mean'))

    # Fill the missing value with newly calculated average
    merged_df[merged_df.columns[col_4]] = merged_df[merged_df.columns[col_4]].fillna(
        merged_df[merged_df.columns[col_4] + '_mean'])

    # drop the mean column
    merged_df.drop(columns=[df.columns[col_4] + '_mean'], inplace=True)

    return merged_df

In [80]:
# call the function with columns: model_year (1), model (2), condition (3) and the target column to fill: odometer (6)
ve = fill_missing(1, 2, 3, 6)

In [81]:
# new total for missing odometer
ve['odometer'].isna().sum()

280

In [82]:
# let's run the function again but this time with only matching model and model_year

def fill_missing_less_entry(col_1, col_2, col_4, df=ve):
    """ 
    This function is designed to fill missing quantitative variables.

    We will be filtering for multiple columns based on index

    And using the average odometer for vehicles matching these 3 columns

    call .info() on your df to gain the index for which columns you would like to filter for

    """
    # Group by the specified columns and calculate the mean for the missing column
    average_odometer = df.groupby([df.columns[col_1], df.columns[col_2]])[
        df.columns[col_4]].mean().reset_index()

    # Merge the averages back onto the original DataFrame
    merged_df = pd.merge(df, average_odometer, on=[df.columns[col_1], df.columns[col_2]],
                         suffixes=('', '_mean'))

    # Fill the missing value with newly calculated average
    merged_df[merged_df.columns[col_4]] = merged_df[merged_df.columns[col_4]].fillna(
        merged_df[merged_df.columns[col_4] + '_mean'])

    # drop the mean column
    merged_df.drop(columns=[df.columns[col_4] + '_mean'], inplace=True)

    return merged_df

In [83]:
# same function this time dont match on condition
ve = fill_missing_less_entry(1, 2, 6)

In [84]:
# new total for missing odometer
ve['odometer'].isna().sum()

77

In [85]:
# let's try just matching just one variable, let's use year instead of model

def fill_missing_less_entry_2(col_1, col_4, df=ve):
    """ 
    This function is designed to fill missing quantitative variables.

    We will be filtering for multiple columns based on index

    And using the average odometer for vehicles matching these 3 columns

    call .info() on your df to gain the index for which columns you would like to filter for

    """
    # Group by the specified columns and calculate the mean for the missing column
    average_odometer = df.groupby([df.columns[col_1]])[
        df.columns[col_4]].mean().reset_index()

    # Merge the averages back onto the original DataFrame
    merged_df = pd.merge(df, average_odometer, on=[df.columns[col_1]],
                         suffixes=('', '_mean'))

    # Fill the missing value with newly calculated average
    merged_df[merged_df.columns[col_4]] = merged_df[merged_df.columns[col_4]].fillna(
        merged_df[merged_df.columns[col_4] + '_mean'])

    # drop the mean column
    merged_df.drop(columns=[df.columns[col_4] + '_mean'], inplace=True)

    return merged_df

In [86]:
# same function this time just year
ve = fill_missing_less_entry_2(1, 6)

# new total for missing odometer
ve['odometer'].isna().sum()

1

In [87]:
# one left, who is this stubborn
ve[ve['odometer'].isna()]

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
45694,18000,1929.0,f-150,good,8.0,gas,NaN,manual,other,silver,NaN,2018-11-18,59,ford


In [88]:
# there was only 1 entry for 1929 so there wasn't a odometer to reference

# I am going to fill with 0. this entry has been viewed before due to the model_year for all I know this truck doesn't even have an odometer 1929.

ve.iloc[45694, 6] = 0

Odometer is complete
___
#### Transmission column
___
No missing values

In [89]:
ve['transmission'].value_counts()

transmission
automatic    46902
manual        2829
other         1794
Name: count, dtype: int64

Looks great
___
#### Type column
___
No missing values

In [90]:
ve['type'].value_counts()

type
SUV            12405
truck          12353
sedan          12154
pickup          6988
coupe           2303
wagon           1541
mini-van        1161
hatchback       1047
van              633
convertible      446
other            256
offroad          214
bus               24
Name: count, dtype: int64

In [91]:
# let's rename some of these, otherwise looks good

# bring back the standardize function
ve = standardize_model_names(ve)

Available columns: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
transmission's unique options: ['automatic' 'manual' 'other']
Available features: Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed', 'make'],
      dtype='object')
Unique models for type: ['SUV' 'pickup' 'sedan' 'truck' 'coupe' 'van' 'hatchback' 'wagon'
 'mini-van' 'convertible' 'other' 'bus' 'offroad']
Changes made:
'SUV' -> 'suv'
'pickup' -> 'truck'
'pickup' -> 'truck'
'SUV' -> 'suv'
'SUV' -> 'suv'
'SUV' -> 'suv'
'pickup' -> 'truck'
'pickup' -> 'truck'
'SUV' -> 'suv'
'pickup' -> 'truck'
'pickup' -> 'truck'
'SUV' -> 'suv'
'SUV' -> 'suv'
'SUV' -> 'suv'
'SUV' -> 'suv'
'SUV' -> 'suv'
'SUV' -> 'suv'
'SUV' -> 'suv'
'SUV' -> 'suv'
'pickup' ->

In [92]:
ve['type'].value_counts()

type
truck          19341
suv            12405
sedan          12154
coupe           2303
wagon           1541
minivan         1161
hatchback       1047
van              633
convertible      446
other            256
offroad          214
bus               24
Name: count, dtype: int64

Looks great
___
#### Paint Color
___
has missing values

In [93]:
ve['paint_color'].value_counts(dropna=False)

paint_color
white     10029
NaN        9267
black      7692
silver     6244
grey       5037
blue       4475
red        4421
green      1396
brown      1223
custom     1153
yellow      255
orange      231
purple      102
Name: count, dtype: int64

In [94]:
# without any context, nothing is named unknown

# fill paint_color with unknown
ve['paint_color'] = ve['paint_color'].fillna('unknown')
ve['paint_color'].value_counts(dropna=False)

paint_color
white      10029
unknown     9267
black       7692
silver      6244
grey        5037
blue        4475
red         4421
green       1396
brown       1223
custom      1153
yellow       255
orange       231
purple       102
Name: count, dtype: int64

Great
___
#### is_4wd column
___
has missing values

In [95]:
# what is going on with 4wd?
ve['is_4wd'].value_counts(dropna=False)

is_4wd
NaN    25953
1.0    25572
Name: count, dtype: int64

In [96]:
# this definitely is not perfect, fill missing 4wd with 0 or not fwd
ve['is_4wd'] = ve['is_4wd'].fillna(0)
ve['is_4wd'].value_counts()

is_4wd
0.0    25953
1.0    25572
Name: count, dtype: int64

4wd done
___
#### Date Posted
___
no missing values, wrong datatype, we will convert and use datetime methods in EDA

In [97]:
# what format is the date in again?
ve['date_posted']

0        2018-06-23
1        2018-10-19
2        2019-02-07
3        2019-03-22
4        2019-04-02
            ...    
51520    2018-10-03
51521    2018-11-14
51522    2018-11-15
51523    2018-07-02
51524    2018-06-05
Name: date_posted, Length: 51525, dtype: object

___
#### Days listed column
___
no missing values

In [98]:
ve['days_listed'].describe()

count    51525.00000
mean        39.55476
std         28.20427
min          0.00000
25%         19.00000
50%         33.00000
75%         53.00000
max        271.00000
Name: days_listed, dtype: float64

In [99]:
ve.info()
ve.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    51525 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     51525 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      51525 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   51525 non-null  object 
 10  is_4wd        51525 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
 13  make          51525 non-null  object 
dtypes: float64(4), int64(2), object(8)
memory usage: 5.5+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
0,9400,2011.0,x5,good,6.0,gas,145000.0,automatic,suv,unknown,1.0,2018-06-23,19,bmw
1,25500,2014.0,f-150,good,6.0,gas,88705.0,automatic,truck,white,1.0,2018-10-19,50,ford
2,5500,2013.0,sonata,like new,4.0,gas,110000.0,automatic,sedan,red,0.0,2019-02-07,79,hyundai
3,1500,2003.0,f-150,fair,8.0,gas,244000.0,automatic,truck,unknown,0.0,2019-03-22,9,ford
4,14900,2017.0,200,excellent,4.0,gas,80903.0,automatic,sedan,black,0.0,2019-04-02,28,chrysler
5,14990,2014.0,300,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15,chrysler
6,12990,2015.0,camry,excellent,4.0,gas,79212.0,automatic,sedan,white,0.0,2018-12-27,73,toyota
7,15990,2013.0,pilot,excellent,6.0,gas,109473.0,automatic,suv,black,1.0,2019-01-07,68,honda
8,11500,2012.0,sorento,excellent,4.0,gas,104174.0,automatic,suv,unknown,1.0,2018-07-16,19,kia
9,9200,2008.0,pilot,excellent,6.0,gas,147191.0,automatic,suv,blue,1.0,2019-02-15,17,honda


In [100]:
# save the csv file to the parent directory
ve.to_csv('../processed_vehicles.csv', index=False)

Data is clean, with the exception of converting datetime.

We have filled all missing entries, looked at outliers, and have standardized the strings